In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb

import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties

plt.rcParams['font.sans-serif'] = ['Times New Roman']  # 用来正常显示中文标签
plt.rcParams["axes.unicode_minus"] = False  # 用来正常显示负号

import sklearn.decomposition
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, AdaBoostRegressor

from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelBinarizer as LB


In [2]:
target_list=['Action','Adventure','Animated','Comedy','Historical','Fantasy','Horror','Science fiction','Documentary']

In [3]:
data=pd.read_csv(r'./data/data_clean_v2.csv')

In [4]:
data

,Extraversion,Observant,Thinking,Judging,Age,way of watching movies,Action,Adventure,Animated,Comedy,...,with_With friends,with_alone,with_with boyfriend/girlfriend,with_with family,Actor/actress,director,advertisements,genres,previews,comment
0,71,64,58,93,23.0,0,5,5,3,4,...,1,0,0,0,1.0,1.0,0.0,1.0,0.0,0.0
1,51,78,76,58,21.0,1,4,2,2,3,...,1,0,0,0,1.0,1.0,0.0,0.0,1.0,0.0
2,57,63,33,39,21.0,0,2,4,5,4,...,0,0,1,0,1.0,0.0,0.0,0.0,1.0,1.0
3,83,16,65,51,23.0,0,2,2,5,5,...,0,0,1,0,1.0,0.0,0.0,1.0,0.0,1.0
4,25,59,68,32,22.0,0,4,4,2,2,...,0,0,1,0,0.0,0.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,60,46,70,68,20.0,0,4,3,2,4,...,0,0,0,1,1.0,0.0,0.0,1.0,1.0,0.0
529,90,35,47,32,26.0,1,2,3,4,2,...,0,1,0,0,0.0,1.0,0.0,1.0,0.0,1.0
530,75,58,60,80,22.0,0,3,2,3,4,...,1,0,0,0,1.0,0.0,0.0,0.0,1.0,1.0
531,82,90,61,70,21.0,0,4,4,4,4,...,0,0,1,0,0.0,0.0,1.0,0.0,1.0,1.0


In [5]:
data_x=data.drop(columns=['Action','Adventure','Animated','Comedy','Historical','Fantasy','Horror','Science fiction','Documentary'],axis=1)
data_x_without=data_x.drop(['Extraversion','Observant','Thinking','Judging'],axis=1)
data_y=data[['Action','Adventure','Animated','Comedy','Historical','Fantasy','Horror','Science fiction','Documentary']]

In [6]:
data_x

,Extraversion,Observant,Thinking,Judging,Age,way of watching movies,Gender_Female,Gender_Male,Gender_Non-binary / third gender,Entertaining,...,with_With friends,with_alone,with_with boyfriend/girlfriend,with_with family,Actor/actress,director,advertisements,genres,previews,comment
0,71,64,58,93,23.0,0,0,1,0,1.0,...,1,0,0,0,1.0,1.0,0.0,1.0,0.0,0.0
1,51,78,76,58,21.0,1,0,1,0,1.0,...,1,0,0,0,1.0,1.0,0.0,0.0,1.0,0.0
2,57,63,33,39,21.0,0,1,0,0,1.0,...,0,0,1,0,1.0,0.0,0.0,0.0,1.0,1.0
3,83,16,65,51,23.0,0,1,0,0,1.0,...,0,0,1,0,1.0,0.0,0.0,1.0,0.0,1.0
4,25,59,68,32,22.0,0,0,1,0,0.0,...,0,0,1,0,0.0,0.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,60,46,70,68,20.0,0,0,1,0,0.0,...,0,0,0,1,1.0,0.0,0.0,1.0,1.0,0.0
529,90,35,47,32,26.0,1,0,1,0,1.0,...,0,1,0,0,0.0,1.0,0.0,1.0,0.0,1.0
530,75,58,60,80,22.0,0,1,0,0,1.0,...,1,0,0,0,1.0,0.0,0.0,0.0,1.0,1.0
531,82,90,61,70,21.0,0,0,1,0,0.0,...,0,0,1,0,0.0,0.0,1.0,0.0,1.0,1.0


In [7]:
data_y

,Action,Adventure,Animated,Comedy,Historical,Fantasy,Horror,Science fiction,Documentary
0,5,5,3,4,1,4,3,5,2
1,4,2,2,3,4,2,1,4,3
2,2,4,5,4,1,5,4,4,4
3,2,2,5,5,2,4,1,3,3
4,4,4,2,2,2,4,2,5,2
...,...,...,...,...,...,...,...,...,...
528,4,3,2,4,3,4,3,5,4
529,2,3,4,2,2,4,3,4,5
530,3,2,3,4,4,4,1,3,4
531,4,4,4,4,4,4,4,4,4


In [8]:
data_x_without

,Age,way of watching movies,Gender_Female,Gender_Male,Gender_Non-binary / third gender,Entertaining,relaxing,funny,express my feelings,educating,...,with_With friends,with_alone,with_with boyfriend/girlfriend,with_with family,Actor/actress,director,advertisements,genres,previews,comment
0,23.0,0,0,1,0,1.0,1.0,0.0,1.0,0.0,...,1,0,0,0,1.0,1.0,0.0,1.0,0.0,0.0
1,21.0,1,0,1,0,1.0,1.0,0.0,1.0,0.0,...,1,0,0,0,1.0,1.0,0.0,0.0,1.0,0.0
2,21.0,0,1,0,0,1.0,1.0,1.0,0.0,0.0,...,0,0,1,0,1.0,0.0,0.0,0.0,1.0,1.0
3,23.0,0,1,0,0,1.0,1.0,1.0,0.0,0.0,...,0,0,1,0,1.0,0.0,0.0,1.0,0.0,1.0
4,22.0,0,0,1,0,0.0,1.0,0.0,0.0,0.0,...,0,0,1,0,0.0,0.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,20.0,0,0,1,0,0.0,1.0,1.0,0.0,1.0,...,0,0,0,1,1.0,0.0,0.0,1.0,1.0,0.0
529,26.0,1,0,1,0,1.0,1.0,0.0,0.0,1.0,...,0,1,0,0,0.0,1.0,0.0,1.0,0.0,1.0
530,22.0,0,1,0,0,1.0,1.0,0.0,1.0,0.0,...,1,0,0,0,1.0,0.0,0.0,0.0,1.0,1.0
531,21.0,0,0,1,0,0.0,1.0,0.0,0.0,0.0,...,0,0,1,0,0.0,0.0,1.0,0.0,1.0,1.0


In [9]:
MM=MinMaxScaler()
data_x[['Extraversion','Observant','Thinking','Judging','Age']]=MM.fit_transform(data_x[['Extraversion','Observant','Thinking','Judging','Age']])
data_x_without['Age']=(data_x_without['Age']-min(data_x_without['Age']))/(max(data_x_without['Age'])-min(data_x_without['Age']))


In [10]:
data_x_without

,Age,way of watching movies,Gender_Female,Gender_Male,Gender_Non-binary / third gender,Entertaining,relaxing,funny,express my feelings,educating,...,with_With friends,with_alone,with_with boyfriend/girlfriend,with_with family,Actor/actress,director,advertisements,genres,previews,comment
0,0.166667,0,0,1,0,1.0,1.0,0.0,1.0,0.0,...,1,0,0,0,1.0,1.0,0.0,1.0,0.0,0.0
1,0.100000,1,0,1,0,1.0,1.0,0.0,1.0,0.0,...,1,0,0,0,1.0,1.0,0.0,0.0,1.0,0.0
2,0.100000,0,1,0,0,1.0,1.0,1.0,0.0,0.0,...,0,0,1,0,1.0,0.0,0.0,0.0,1.0,1.0
3,0.166667,0,1,0,0,1.0,1.0,1.0,0.0,0.0,...,0,0,1,0,1.0,0.0,0.0,1.0,0.0,1.0
4,0.133333,0,0,1,0,0.0,1.0,0.0,0.0,0.0,...,0,0,1,0,0.0,0.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,0.066667,0,0,1,0,0.0,1.0,1.0,0.0,1.0,...,0,0,0,1,1.0,0.0,0.0,1.0,1.0,0.0
529,0.266667,1,0,1,0,1.0,1.0,0.0,0.0,1.0,...,0,1,0,0,0.0,1.0,0.0,1.0,0.0,1.0
530,0.133333,0,1,0,0,1.0,1.0,0.0,1.0,0.0,...,1,0,0,0,1.0,0.0,0.0,0.0,1.0,1.0
531,0.100000,0,0,1,0,0.0,1.0,0.0,0.0,0.0,...,0,0,1,0,0.0,0.0,1.0,0.0,1.0,1.0


In [11]:
def MAPE(true, pred):
    nonzero = np.flatnonzero(true)
    true = true.ravel()[nonzero]
    pred = pred.ravel()[nonzero]
    diff = np.abs(np.array(true) - np.array(pred))
    mape = np.nanmean(diff / true)
    return mape

In [12]:
error_sum_rfr=0
error_rfr=[]
error_sum_without_rfr=0
error_without_rfr=[]
for i in target_list:
    x_train, x_test ,y_train,y_test=train_test_split(data_x,data_y[i],test_size=0.2, random_state=50,shuffle=True)
    rfr = RandomForestRegressor()
    rfr.fit(x_train, y_train)
    rfr_y_predict = rfr.predict(x_test)
    error_sum_rfr+=MAPE(y_test,rfr_y_predict)
    error_rfr.append(MAPE(y_test,rfr_y_predict))
    x_train_wt, x_test_wt ,y_train_wt,y_test_wt=train_test_split(data_x_without,data_y[i],test_size=0.2, random_state=50,shuffle=True)
    rfr_without = RandomForestRegressor()
    rfr_without.fit(x_train_wt, y_train_wt)
    rfr_y_without_predict = rfr_without.predict(x_test_wt)
    error_sum_without_rfr+=MAPE(y_test_wt,rfr_y_without_predict)
    error_without_rfr.append(MAPE(y_test_wt,rfr_y_without_predict))

In [13]:
print(error_sum_rfr/9)
print(error_sum_without_rfr/9)

0.3980276998961578
0.3836489943381298


In [14]:
error_sum_ad=0
error_ad=[]
error_sum_without_ad=0
error_without_ad=[]
for i in target_list:
    x_train, x_test ,y_train,y_test=train_test_split(data_x,data_y[i],test_size=0.2, random_state=50,shuffle=True)
    ad = AdaBoostRegressor()
    ad.fit(x_train, y_train)
    ad_y_predict = ad.predict(x_test)
    error_sum_ad+=MAPE(y_test,ad_y_predict)
    error_ad.append(MAPE(y_test,ad_y_predict))
    x_train_wt, x_test_wt ,y_train_wt,y_test_wt=train_test_split(data_x_without,data_y[i],test_size=0.2, random_state=50,shuffle=True)
    ad_without = AdaBoostRegressor()
    ad_without.fit(x_train_wt, y_train_wt)
    ad_y_without_predict = ad_without.predict(x_test_wt)
    error_sum_without_ad+=MAPE(y_test_wt,ad_y_without_predict)
    error_without_ad.append(MAPE(y_test_wt,ad_y_without_predict))

In [15]:
print(error_sum_ad/9)
print(error_sum_without_ad/9)

0.3992705245920495
0.3987569560253216


In [38]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',  # 设置提升类型
    'objective': 'mape',  # 目标函数
    'metric': {'mape'},  # 评估函数
    'num_leaves': 50,  # 叶子节点数
    'learning_rate': 0.02,  # 学习速率
    'feature_fraction': 1,  # 建树的特征选择比例
    'bagging_fraction': 1,  # 建树的样本采样比例
    'bagging_freq': 5,  # k 意味着每 k 次迭代执行bagging
    'verbose': 1  # <0 显示致命的, =0 显示错误 (警告), >0 显示信息
}
error_sum_lgb=0
error_lgb=[]
error_sum_without_lgb=0
error_without_lgb=[]
for i in target_list:
    x_train, x_test ,y_train,y_test=train_test_split(data_x,data_y[i],test_size=0.2, random_state=50,shuffle=True)
    lgb_train = lgb.Dataset(x_train, y_train)
    gbm = lgb.train(params, lgb_train, num_boost_round=100)
    lgb_y_pred = gbm.predict(x_test, num_iteration=gbm.best_iteration)
    error_sum_lgb+=MAPE(y_test,lgb_y_pred)
    error_lgb.append(MAPE(y_test,lgb_y_pred))
    x_train_wt, x_test_wt ,y_train_wt,y_test_wt=train_test_split(data_x_without,data_y[i],test_size=0.2, random_state=50,shuffle=True)
    lgb_train_wt = lgb.Dataset(x_train_wt, y_train_wt)
    gbm_wt = lgb.train(params, lgb_train_wt, num_boost_round=100)
    lgb_y_pred_wt = gbm_wt.predict(x_test_wt, num_iteration=gbm_wt.best_iteration)
    error_sum_without_lgb+=MAPE(y_test_wt,lgb_y_pred_wt)
    error_without_lgb.append(MAPE(y_test_wt,lgb_y_pred_wt))

[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000163 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 309
[LightGBM] [Info] Number of data points in the train set: 426, number of used features: 39
[LightGBM] [Info] Start training from score 3.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000143 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 309
[LightGBM] [Info] Number of data points in the train set: 426, number of used features: 39
[LightGBM] [Info] Start training from score 3.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000153 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 309
[LightGBM] [Info] Number of data points in the train set: 426, number of used features: 39
[LightGBM] [Info] Start training from score 2.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [39]:
print(error_sum_lgb/9)
print(error_sum_without_lgb/9)

0.34872153069692635
0.3342270324865535


In [40]:
print(error_rfr)
print(error_without_rfr)
print(error_ad)
print(error_without_ad)
print(error_lgb)
print(error_without_lgb)

[0.29588473520249226, 0.3204672897196262, 0.3744302959501557, 0.33374922118380057, 0.38699376947040504, 0.3684020768431982, 0.7058504672897196, 0.39745742471443396, 0.3990140186915888]
[0.24879556074766357, 0.2875060747663551, 0.36106056223112304, 0.3273178219106957, 0.3877479231568016, 0.3764310488058152, 0.6942218328141224, 0.3772538940809969, 0.39250623052959505]
[0.296063698328951, 0.3231800437590797, 0.37736110574850185, 0.34103385731978314, 0.3831520082331621, 0.36668675672526624, 0.7549856390529932, 0.3646111580448649, 0.3863604541158435]
[0.29665346951265137, 0.3262040334085523, 0.38294645963908025, 0.3367567614427228, 0.38617599890974763, 0.36767268375482426, 0.7604212885783604, 0.3559887700201396, 0.375993138961816]
[0.2533016625823991, 0.2801947716575992, 0.3380664280525985, 0.3283328526753244, 0.34772156709151747, 0.3441047626686998, 0.5346901560637161, 0.3437989196609899, 0.3682826558194922]
[0.22105128618493083, 0.27527219582243745, 0.3163203338016652, 0.3016086623882042,

In [41]:
res_rfr=[error_rfr,error_without_rfr]
res_rfr=pd.DataFrame(np.array(res_rfr).T)
res_ad=[error_ad,error_without_ad]
res_ad=pd.DataFrame(np.array(res_ad).T)
res_lgb=[error_lgb,error_without_lgb]
res_lgb=pd.DataFrame(np.array(res_lgb).T)

In [42]:
res_rfr.columns=['with','without']
res_ad.columns=['with','without']
res_lgb.columns=['with','without']
res_rfr.index=target_list
res_ad.index=target_list
res_lgb.index=target_list

In [43]:
res_rfr

,with,without
Action,0.295885,0.248796
Adventure,0.320467,0.287506
Animated,0.374430,0.361061
Comedy,0.333749,0.327318
Historical,0.386994,0.387748
Fantasy,0.368402,0.376431
Horror,0.705850,0.694222
Science fiction,0.397457,0.377254
Documentary,0.399014,0.392506


In [44]:
res_ad

,with,without
Action,0.296064,0.296653
Adventure,0.323180,0.326204
Animated,0.377361,0.382946
Comedy,0.341034,0.336757
Historical,0.383152,0.386176
Fantasy,0.366687,0.367673
Horror,0.754986,0.760421
Science fiction,0.364611,0.355989
Documentary,0.386360,0.375993


In [45]:
res_lgb

,with,without
Action,0.253302,0.221051
Adventure,0.280195,0.275272
Animated,0.338066,0.316320
Comedy,0.328333,0.301609
Historical,0.347722,0.340115
Fantasy,0.344105,0.345288
Horror,0.534690,0.510535
Science fiction,0.343799,0.338113
Documentary,0.368283,0.359740


In [46]:
res_rfr.to_csv(r'.\data\res_rfr.csv',index=True)
res_ad.to_csv(r'.\data\res_ad.csv',index=True)
res_lgb.to_csv(r'.\data\res_lgb.csv',index=True)